# Object Following - Live Demo


In [1]:
import time
from jetbot import Camera, Robot, ObjectDetector, bgr8_to_jpeg
from IPython.display import display
import ipywidgets.widgets as widgets
import outsourcing_nechl as myhelp
import cv2
import numpy as np
from importlib import reload
from ipywidgets import TwoByTwoLayout
import nechlBot
reload(nechlBot)

#only for the obstacle avoidance
import torch
import torchvision
import torch.nn.functional as F

In [2]:
model_ob = torchvision.models.alexnet(pretrained=False)
model_ob.classifier[6] = torch.nn.Linear(model_ob.classifier[6].in_features, 2)

In [3]:
model_ob.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [4]:
device = torch.device('cuda')
model_ob = model_ob.to(device)

In [5]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [6]:
return_load_things = myhelp.load_things()
items = return_load_things[0]
font = return_load_things[1]
color = return_load_things[2]
fontScale = return_load_things[3]
thickness = return_load_things[4]

### Compute detections on single camera image

In [7]:
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [8]:
camera = Camera.instance(width=300, height=300, fps=5)

In [9]:
robot = nechlBot.NechlBot()

In [10]:
detections = myhelp.detectt(model,camera)

In [11]:
reload(myhelp)
image_widget = widgets.Image(format='jpeg', width=300, height=300)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')


return_create_widgets=myhelp.create_widgets(detections)
button_box = return_create_widgets[0]
detections_widget = return_create_widgets[1]
button_stop_stream = return_create_widgets[2]
button_start_stream = return_create_widgets[3]
label_widget = return_create_widgets[4]
search_for_widget = return_create_widgets[5]
button_search_box = return_create_widgets[6]
button_start_search = return_create_widgets[7]
button_stop_search = return_create_widgets[8]
def cam_stop(change):
    camera.unobserve_all()

def cam_start(change):
    camera.unobserve_all()
    camera.observe(execute, names="value")
def start_moving(change):
    robot.searching = True

def restart_searching(change):
    robot.searching_status = False
    robot.item_found = False
    robot.round_search = 0
def force_stop(change):
    robot.searching_status = False
    robot.item_found = True

button_stop_stream.on_click(cam_stop)
button_start_stream.on_click(cam_start)
button_start_search.on_click(restart_searching)
button_stop_search.on_click(force_stop)

In [12]:
image_number = 0
object_number = 0
try:
    det = detections[image_number][object_number]

    for i in range(len(items)):
        if str(det["label"]) == items[str(i)][1]:
            #print("it is a", items[str(i)][2])
            item_detected = items[str(i)][2]
except IndexError as e:
    pass


### Loop through the process for a moving image

In [13]:
myhelp.fill_label(items, label_widget)
myhelp.fill_label(items, search_for_widget)
left_col = widgets.HBox([image_widget, blocked_slider])
right_col = widgets.VBox([label_widget,button_box, search_for_widget, button_search_box])

t2x2 = TwoByTwoLayout(
    top_left = left_col,
    top_right = right_col

)


In [20]:
robot.searching_status = False
display(t2x2)
width = int(image_widget.width)
height = int(image_widget.height)
def execute(change):
    image = change['new']
    offset_text = 10
    x = preprocess(image)
    y = model_ob(x)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    blocked_slider.value = prob_blocked

    
    
    detections = model(image)
    detection_names = myhelp.get_names(detections, items)
    try:
        first_detection=detections[0][0]
        for i in range(len(items)):
            if str(first_detection["label"]) == items[str(i)][1]:
                item_detected = items[str(i)][2]
    except IndexError as e:
        pass
    
    for det in detections[0]:
        for i in range(len(items)):
            if str(det["label"]) == items[str(i)][1]:
                item_detected = items[str(i)][2]
                item_chance = str(round((det["confidence"]*100),1))

        bbox = det['bbox']
        
        if (item_detected != search_for_widget.value) and (item_detected != label_widget.value):
            new_img = cv2.rectangle(image,  (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            new_img = cv2.rectangle(new_img,(int(width * bbox[0]), int(height* bbox[1])), (int(width*bbox[2]), int(height * bbox[1]+12)), (255, 0, 0), -1) 
            cv2.putText(new_img, item_detected+"  "+item_chance+"%",(int(width*bbox[0]+5),int(height*bbox[1]+offset_text)), font, 0.3, color,1)
        else: 
            pass
    return_check_selected_label = myhelp.check_selected_label(items, label_widget.value, detections)
    if return_check_selected_label is not None:
        
        bbox = return_check_selected_label['bbox']
        offset_text = 10        
                
        item_detected = label_widget.value
        item_chance = str(round((return_check_selected_label["confidence"]*100),1))
        new_img=cv2.rectangle(image,  (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
        new_img=cv2.rectangle(new_img,(int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[1]+12)), (0, 255, 0),-1)
        cv2.putText(new_img,item_detected+"  "+item_chance+"%" ,(int(width* bbox[0])+2,int(height*bbox[1])+offset_text), font,0.3,(0,0,0),1)
    elif return_check_selected_label is None:
        new_img=image
    else:
        pass
    
    
    ###this part draws the box around the item that has to be detected
    return_get_names=(myhelp.get_names(detections,items)) #this revonverts the labels in digits to the string again, which later is displayed
    return_get_names_bbox = myhelp.check_selected_label(items, search_for_widget.value, detections) #check if the search field item is currently avaiavle

    if return_get_names_bbox is None and search_for_widget.value != "background":
        pass
    if return_get_names_bbox is not None:
        bbox = return_get_names_bbox['bbox']
        item_detected = search_for_widget.value
        item_info = [item_detected,return_get_names_bbox['label']]
        item_chance = str(round((return_get_names_bbox['confidence']*100),1))
        new_img=cv2.rectangle(new_img,  (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
        new_img=cv2.rectangle(new_img,(int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[1]+12)), (0, 0, 255),-1)
        cv2.putText(new_img,item_detected+"  "+item_chance+"%" ,(int(width* bbox[0])+2,int(height*bbox[1])+offset_text), font,0.3,(0,0,0),1)

    
    ###thanks to item_found, we only need one frame with positive detection and the robot stops, that is very useful.
    if (search_for_widget.value not in detection_names) and (robot.item_found == False) and (search_for_widget.value is not 'background'):
        if prob_blocked > 0.51:
            robot.backward(0.3)
            time.sleep(0.1)
            robot.rotate_left(22.5)
        elif prob_blocked <0.5:
            robot.searching_around()
            robot.searching_status = True
            
    elif (search_for_widget.value in detection_names) or (robot.item_found == True):
        robot.searching_status = False
        robot.item_found = True
   

     
    # update image widget
    image_widget.value = bgr8_to_jpeg(new_img)
    time.sleep(0.1)
    
execute({'new': camera.value})



TwoByTwoLayout(children=(HBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00…

robot.searching_status 


In [15]:
#dir(dd)

In [16]:
#dd.trait_names()

from traitlets import HasTraits,Bool
import time
class Test(HasTraits):
    status = Bool()

    def searching_status_watcher(self, status):
        if status["old"] == False and status["new"] == True:
            robot.forward(0.2)
        elif status['old'] == True and status['new'] == False:
            robot.stop()
    
    
test = Test()
test.observe(test.searching_status_watcher,names=['status',])


In [17]:
test.status = False

NameError: name 'test' is not defined

In [ ]:
r.item_searching